In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/selfeval/pope_adversarial')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [ ]:
df.head()

,img_path,question,is_correct,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Question: Is there a baseball glove in the im...,[tensor(1)],{'embedding': [[[-0.0201416 -0.00386047 -0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Question: Is there a cell phone in the image?...,[tensor(1)],{'embedding': [[[-0.01843262 0.00072479 0.01...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Question: Is there a boat in the image?\nAnsw...,[tensor(1)],{'embedding': [[[-0.01843262 0.00072479 0.01...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Question: Is there a suitcase in the image?\n...,[tensor(1)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Question: Is there a handbag in the image?\nA...,[tensor(0)],{'embedding': [[[-0.01843262 0.00072479 0.01...,None


In [5]:
df.iloc[0]['question']

['Question: Is there a baseball glove in the image?\nAnswer the question using a single word or phrase.\n Proposed Answer: No\n Is the proposed answer: A.True \nB.False\n']

In [ ]:
is_correct=df['is_correct'].apply(lambda x: x[0].item()).to_list()
is_correct[:20]

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]

In [7]:
resps=df['most_likely'].apply(lambda x: x['response'][0]).to_list()
resps[:10]

['B.False',
 'A True',
 'A True',
 'A',
 'A True',
 'B.False',
 'B.False',
 'B.False',
 'A.True',
 'B.False']

In [8]:
from answer_judge.SelfEval import SelfEvalJudge
judge=SelfEvalJudge()

self_check_result=[]
for i,x in enumerate(resps):
    if x.lower()[0]=='a':
        self_check_result.append(1)
    else:
        self_check_result.append(0)
    
self_check_result=np.array(self_check_result)

In [9]:
self_check_result.shape

(2000,)

In [10]:
print(f'There are {len(self_check_result)} responses. {sum(self_check_result==1)} answer correctly and {sum(self_check_result==0)} answer wrongly')

There are 2000 responses. 912 answer correctly and 1088 answer wrongly


In [11]:
print(roc_auc_score(is_correct,self_check_result))

0.6014692885550155
